In [9]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options


from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
base_url = "https://stats.ncaa.org/"

In [ ]:
games = pd.read_csv('games_2023-24.csv',header=0).GameID.drop_duplicates().tolist()
games

In [8]:
len(games)

5899

In [15]:
# 1:39:34 runtime for 5899 games

df = pd.DataFrame()
errorGames = []
errors = []

options = Options()
#options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
for gameId in tqdm(games, total=len(games)):
    try:
        driver.get(base_url + f'contests/{gameId}/box_score')

        soup = BeautifulSoup(driver.page_source, "html.parser")
            
        shotsList = soup.find_all('circle')[3:-4]

        shots = pd.DataFrame([shot.attrs for shot in shotsList])
        shots[['Period','player_id','team_id','shot','ShotMade']] = pd.DataFrame(shots['class'].to_list())

        shots['text']    = pd.DataFrame([shot.text for shot in shotsList])
        shots['Team']    = shots['text'].apply(lambda string: re.findall('\\((.+)\\)',string)[0])
        shots['Player']  = shots['text'].apply(lambda string: re.findall('by (.+)\\(',string)[0])
        shots['Time']    = shots['text'].apply(lambda string: re.findall('... (.*) :',string)[0])
        shots['game_id'] = gameId

        shots.drop(columns=['r','style','class','shot','text'], inplace=True)

        # Sort the events based on period and time remaining in quarter; this is mainly due to overtime periods being out of order
        shots.sort_values(['Period','Time'],ascending=[True,False],inplace=True)
        shots.reset_index(drop=True,inplace=True)

        df = pd.concat([df,shots])
    except Exception as e:
        errorGames.append(gameId)
        errors.append(e)
df.to_csv('shots_2023_24.csv',index=False)

100%|██████████| 5899/5899 [1:39:34<00:00,  1.01s/it]  


In [28]:
driver.quit()

In [ ]:
len(errorGames)

In [31]:
pd.DataFrame(errorGames,columns=['gamesWithoutShotCharts']).to_csv('missing_shot_chart_2023_24.csv',index=False)

In [32]:
df

,cx,cy,id,Period,player_id,team_id,ShotMade,Team,Player,Time,game_id
0,883.5999999999999,140,play_2552179163,period_1,player_767381654,team_2707,made,Kansas City,Dominique Phillips,09:42:00,3970221
1,188,235,play_2552179165,period_1,player_767381672,team_72,made,Bradley,Kaylen Nelson,09:09:00,3970221
2,178.6,120,play_2552179168,period_1,player_767381676,team_72,made,Bradley,Ruba Abo Hashesh,08:28:00,3970221
3,65.80000000000001,55,play_2552179174,period_1,player_767381676,team_72,missed,Bradley,Ruba Abo Hashesh,08:02:00,3970221
4,846,220,play_2552179176,period_1,player_767381659,team_2707,missed,Kansas City,Tamia Ugass,07:49:00,3970221
...,...,...,...,...,...,...,...,...,...,...,...
124,197.4,295,play_2554033170,period_4,player_767506497,team_506547,missed,UNT Dallas,ZyUnn Cormier,02:02:00,3970551
125,639.2,265,play_2554033172,period_4,player_767506438,team_199,missed,Tex. A&amp;M-Commerce,Dorian Norris,01:54:00,3970551
126,827.2,435,play_2554033176,period_4,player_767506448,team_199,missed,Tex. A&amp;M-Commerce,Olivia Russell,01:31:00,3970551
127,225.6,65,play_2554033193,period_4,player_767506485,team_506547,missed,UNT Dallas,Breniya Arnold,01:07:00,3970551
